### Virtual Private Cloud (VPC)
##### Delete Default VPCs

In [ ]:
import botoplus.botoplus as botoplus

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:

        print(' - '+region['RegionName'])
        ec2_region = session.client('ec2', region_name = region['RegionName'])
        paginator = ec2_region.get_paginator('describe_vpcs')
        response_iterator = paginator.paginate()
        for page in response_iterator:
            if len(page['Vpcs']) > 0:
                for item in page['Vpcs']:
                    if item['IsDefault'] is True:

                        paginator2 = ec2_region.get_paginator('describe_internet_gateways')
                        response_iterator2 = paginator2.paginate()
                        for page2 in response_iterator2:
                            for item2 in page2['InternetGateways']:
                                if len(page2['InternetGateways']) > 0:
                                    if item2['Attachments'][0]['VpcId'] == item['VpcId']:
                                        try:
                                            ec2_region.detach_internet_gateway(
                                                InternetGatewayId=item2['InternetGatewayId'],
                                                VpcId=item['VpcId']
                                            )
                                            ec2_region.delete_internet_gateway(
                                                InternetGatewayId=item2['InternetGatewayId']
                                            )
                                        except:
                                            print('ERROR '+str(item2))

                        paginator3 = ec2_region.get_paginator('describe_subnets')
                        response_iterator3 = paginator3.paginate()
                        for page3 in response_iterator3:
                            for item3 in page3['Subnets']:
                                if len(page3['Subnets']) > 0:
                                    if item3['VpcId'] == item['VpcId']:
                                        try:
                                            ec2_region.delete_subnet(
                                                SubnetId=item3['SubnetId']
                                            )
                                        except:
                                            print('ERROR '+str(item3))

                        try:
                            ec2_region.delete_vpc(
                                VpcId=item['VpcId']
                            )
                        except:
                            print('ERROR '+str(item))
                            pass